In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(output):
    return output * (1 - output)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(output):
    return 1 - output ** 2

class MLP:
    def __init__(self, layer_sizes, learning_rate=0.1):
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        
        # Initialize weights with small random numbers
        self.weights = []
        self.biases = []
        for i in range(len(layer_sizes) - 1):
            w = np.random.randn(layer_sizes[i], layer_sizes[i+1]) * 0.1
            b = np.zeros((1, layer_sizes[i+1]))
            self.weights.append(w)
            self.biases.append(b)
    
    def forward(self, X):
        self.a1 = X
        self.z2 = np.dot(self.a1, self.weights[0]) + self.biases[0]
        self.a2 = tanh(self.z2)  # hidden layer with tanh
        
        self.z3 = np.dot(self.a2, self.weights[1]) + self.biases[1]
        self.a3 = sigmoid(self.z3)  # output layer sigmoid for binary
        
        return self.a3
    
    def backward(self, X, y, output):
        m = y.shape[0]
        
        # Output layer error
        delta3 = (output - y) * sigmoid_derivative(output)
        
        # Hidden layer error
        delta2 = delta3.dot(self.weights[1].T) * tanh_derivative(self.a2)
        
        # Gradients
        dw2 = self.a2.T.dot(delta3) / m
        db2 = np.sum(delta3, axis=0, keepdims=True) / m
        
        dw1 = X.T.dot(delta2) / m
        db1 = np.sum(delta2, axis=0, keepdims=True) / m
        
        # Update weights and biases
        self.weights[1] -= self.learning_rate * dw2
        self.biases[1] -= self.learning_rate * db2
        
        self.weights[0] -= self.learning_rate * dw1
        self.biases[0] -= self.learning_rate * db1
        
    def train(self, X, y, epochs=10000, print_every=1000):
        for epoch in range(1, epochs+1):
            output = self.forward(X)
            self.backward(X, y, output)
            
            if epoch % print_every == 0:
                loss = -np.mean(y * np.log(output + 1e-9) + (1 - y) * np.log(1 - output + 1e-9))
                print(f"Epoch {epoch}, Loss: {loss:.4f}")
    
    def predict(self, X):
        output = self.forward(X)
        return (output > 0.5).astype(int)

# XOR Example
if __name__ == "__main__":
    X = np.array([
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]
    ])
    y = np.array([[0], [1], [1], [0]])
    
    mlp = MLP(layer_sizes=[2, 4, 1], learning_rate=0.1)
    mlp.train(X, y, epochs=10000, print_every=1000)
    
    preds = mlp.predict(X)
    print("Predictions:")
    print(preds)
